In [2]:
import os
from collections import defaultdict
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

In [1]:
def pre_process(text):
    '''
    Lowercase, TOkenize (Stanford CoreNLP)
    '''
    text = text.lower()
    # Ref : https://github.com/Lynten/stanford-corenlp
    nlp = StanfordCoreNLP(r'..\pkgs\stanford-corenlp-full-2016-10-31')
    result = nlp.word_tokenize(text)
    return result



def train(model,train_loader,valid_loader,test_loader, numEpochs=5):
    '''
    Args:

    Ret: 
    '''
    model.train()

    for epoch in range(numEpochs):
        avg_loss = 0.0
        for batch_num, (feats, labels) in enumerate(train_loader):
            feats,labels = feats.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(feats)

            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()

            avg_loss += loss.item()

            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0

            torch.cuda.empty_cache()
            del feats
            del labels

    torch.save(model.state_dict(), "jessi_A_"+str(epoch)+".pt")

    val_loss, val_acc = validate(model, valid_loader)
    test_loss, test_acc = test(model, test_loader, epoch)
    print('Epoch{:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}\tTest Loss: {:.4f}\tTest Accuracy: {:.4f}'.format(epoch, val_loss, val_acc, test_loss, test_acc))



def validate(model, valid_loader):
    '''
    Args:

    Ret:
    '''

    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(valid_loader):
        feats,labels = feats.to(device), labels.to(device)
        output = model(feats)

        _, pred_labesl = torch.max(output, dim=1)
        pred_labels = pred_labels.view(-1)

        #loss = criterion(output, labels.long())
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        #test_loss.extend([loss.item()]*feats.size()[0])

        #torch.cuda.empty_cache()
        del feats
        del labels

    model.train()
    #return np.mean(test_loss), accuracy/total
    return 0, accuracy/total


def test(model, test_loader, epoch):
    '''
    Args:

    Ret:
    '''

    model.eval()
    test_loss = []
    accuracy = 0
    total = 0
    
    outcomes = []
    
    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)[0]
        
        
        _, pred_labels = torch.max(outputs, dim=1)
        #loss = criterion(outputs, labels.long())
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        #test_loss.extend([loss.item()]*feats.size()[0])
        
        pred_labels = pred_labels.view(-1)
        pred_labels = pred_labels.cpu()
        pred_labels = np.array(pred_labels)
        outcomes += [ [test_loader.dataset.samples[batch_num*BATCH_SIZE+i][0].split('/')[-1],
            pred_labels[i]] for i in range(len(pred_labels)) ]
        
        del feats
        del labels

    model.train()
    
    f = open("Suggestion"+str(epoch)+".csv", "w", newline="")
    writer = csv.writer(f)
    writer.writerows(outcomes)
    f.close()
    
    #return np.mean(test_loss), accuracy/total
    return 0, accuracy/total

In [ ]:
# class SuggestionDataset(Dataset):
#     def __init__(self, filename):
#         self.id_map = ID_Mapping(filename)
        

#     def __len__(self):
#         return len(self.id_map)
        

#     def __getitem__(self, index):
        
        
import pathlib

        

        
ID_Mapping('as')